In [ ]:
%load_ext autoreload 
%autoreload 2

write abundance predictions as json files

In [ ]:
from utils.helper_functions import * 
import matplotlib.pyplot as plt

benchmark_dirs = ["Connecticut"]
ref_sets = [["Connecticut", "USA", "North_America", "Global"]]
start_dates = ["2020-01-01_till_2022-01-01", "2020-06-01_till_2022-01-01", "2021-01-01_till_2022-01-01", "2021-06-01_till_2022-01-01"]
abundances = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]

for ref_sts in  ref_sets:
    output_results_to_json_2_dirs( "0.1", ref_sts, start_dates, abundances, "AY.103_sequence")
    output_results_to_json_2_dirs( "0.1", ref_sts, start_dates, abundances, "AY.103_sequence", "VOC", "delta")

read abundance prediction

In [ ]:
results = dict()
results_who = dict()

for dir_name in benchmark_dirs:
    results[dir_name] = dict()
    # load results
    with open("results.json") as json_file:
        results[dir_name] = json.loads(json_file.read())
    with open("results_who.json") as json_file:
        results_who[dir_name] = json.loads(json_file.read())

calculate absolute errors

In [ ]:
ref_sets_dict = dict()

for dir, ref_set_list in zip(benchmark_dirs, ref_sets):
    ref_sets_dict[dir] = ref_set_list

absolute_errors = calculate_absolute_errors_af(results, start_dates, abundances, ref_sets_dict, benchmark_dirs)
absolute_errors_who = calculate_absolute_errors_af(results_who, start_dates, abundances, ref_sets_dict, benchmark_dirs)

generate scatter plot

In [ ]:
def plot_with_scatterplots_custom(abundances, dates, results, dirs_and_refs, bench_dir, special_plot_name = None):

    fig, ax = plt.subplots(2, 2, figsize=(20, 20))
    # fig.set_dpi(2000)

    for ref_set, tuple in zip(dirs_and_refs[bench_dir],[[0,0], [0,1], [1,0], [1,1]]):
        i = tuple[0]
        j = tuple[1]
        ax[i][j].grid()
        for date in dates:
            # create a new figure
            ax[i][j].scatter(abundances, [results[bench_dir][ref_set][date][str(abundance)] for abundance in abundances], label= date.replace("_", " "), s = 120, alpha= 0.7)
            ax[i][j].set_xlabel("Simulated abundance", fontsize = 17)
            ax[i][j].set_ylabel("Predicted abundance", fontsize = 17)
            ax[i][j].set_xscale('log')
            ax[i][j].set_title("{}".format(ref_set.replace("_", " ")), fontweight='bold', fontsize = 19)
            
        # plot x=y line black and dashed
        ax[i][j].plot(abundances, abundances, color='black', linestyle='dashed', label = "True abundance")
        ax[i][j].legend(fontsize = 13)
        ax[i][j].set_yticks([0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100])
    
        ax[i][j].tick_params(axis='both', labelsize=15)

            # save figure as pdf with tight layout
    plt.subplots_adjust(hspace=0.19)
    if special_plot_name == None:
        plt.savefig("figures/scatter_plot.pdf", bbox_inches='tight')
    else:
        plt.savefig("figures/{}.pdf".format(special_plot_name), bbox_inches='tight')


    return 


plot_with_scatterplots_custom(abundances, start_dates, results_who, ref_sets_per_dir, "Connecticut", "scatter_plot_custom")

In [ ]:
from utils.plotting_functions import *
if os.path.isdir("figures") != True:
    os.mkdir("figures")

ref_sets_per_dir = dict()
for dir, ref_set_l in zip(benchmark_dirs, ref_sets):
    ref_sets_per_dir[dir] = ref_set_l
plot_with_scatterplots_generic(abundances, start_dates, results, ref_sets_per_dir, "Connecticut")
plot_with_scatterplots_generic(abundances, start_dates, results_who, ref_sets_per_dir, "Connecticut", "scatter_plot_who")



output dataset info

In [ ]:
from utils.helper_functions import * 

dirs = ["Connecticut", "USA", "North_America", "Global"]
start_dates = ["2020-01-01_till_2022-01-01", "2020-06-01_till_2022-01-01", "2021-01-01_till_2022-01-01", "2021-06-01_till_2022-01-01"]

for ref_set in dirs:
    for dates in start_dates:
        path = "../../../data/Timeframe_Experiments/HPC/reference_sets/{}/{}".format(ref_set, dates)
        selection_path = "../../../data/Timeframe_Experiments/HPC/reference_sets/{}/{}/processed".format(ref_set, dates)
        output_dataset_info("AY.103", path, selection_path)

# merge csv per directory
for dir in dirs:
    merge_csv_from_subdirectory("../../../data/Timeframe_Experiments/HPC/reference_sets/{}".format(dir, dates), "Reference set", True)